In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from functools import reduce

from utils import *
import graphing

In [2]:
events = pd.read_csv("datasets/events_England.csv")
teams = pd.read_csv("datasets/teams.csv")
matches = pd.read_csv("datasets/games.csv")

In [3]:
teams[teams.name.str.contains("Liverpool")]

,city,name,wyId,officialName,area,type
77,Liverpool,Liverpool,1612,Liverpool FC,"{'name': 'England', 'id': '0', 'alpha3code': '...",club


In [4]:
matches[matches.game_id==2499943]

,Unnamed: 0,game_id,competition_id,season_id,game_date,game_day,home_team_id,away_team_id
151,151,2499943,364,181150,2018-01-14 16:00:00,23,1612,1625


In [5]:
teams.index = teams.wyId

In [29]:
match = events.loc[events.matchId==2499943, :].copy()
match.shape

(1876, 17)

In [30]:
names = teams.name.to_dict()
match["team"] = match.teamId.map(names)

In [31]:
tags_na = match.tagsList.isna()
match.loc[tags_na, "tagsList"] = "[]"

In [32]:
match['successful_action'] = None
success = match.tagsList.str.contains("1801")
fail = match.tagsList.str.contains("1802")
match.loc[success, 'successful_action'] = True
match.loc[fail, 'successful_action'] = False

In [33]:
match.loc[:,"possession"] = None

In [34]:
successful_pass = (match.eventName == "Pass") & (match.successful_action == True)
match.loc[successful_pass, 'possession'] = match[successful_pass].team

In [35]:
unsuccessful_pass = (match.eventName == "Pass") & (match.successful_action == False)
match.loc[unsuccessful_pass, 'possession'] ="Turnover"

In [36]:
won_duels = (match.eventName=="Duel") & (match.successful_action == True)
match.loc[won_duels, 'possession'] = match.loc[won_duels, 'team']

In [38]:
unsuccessful_clearances = (match.subEventName == "Clearance") & (match.successful_action == False)
match.loc[unsuccessful_clearances, "possession"] = "Turnover"

In [39]:
restarts = match.eventName == 'Free Kick'
match.loc[restarts, 'possession'] = match.loc[restarts, 'team']

In [40]:
match['time'] = (match.eventSec.astype(int)//60).astype(str)+":"+(match.eventSec%60).astype("int").astype(str)

In [41]:
match.possession = match.possession.ffill()

In [42]:
match.eventId

378660    8
378661    8
378662    8
378663    8
378664    8
         ..
380531    3
380532    1
380533    1
380534    8
380535    8
Name: eventId, Length: 1876, dtype: int64

In [43]:
match.eventId.shift(-1) == match.eventId

378660     True
378661     True
378662     True
378663     True
378664    False
          ...  
380531    False
380532     True
380533    False
380534     True
380535    False
Name: eventId, Length: 1876, dtype: bool

In [44]:
new_phases = match.possession

In [45]:
match.time.shift(1)

378660      NaN
378661      0:2
378662      0:6
378663     0:10
378664     0:14
          ...  
380531    49:10
380532    49:43
380533    49:48
380534    49:48
380535    49:50
Name: time, Length: 1876, dtype: object

In [52]:
new_phases = (match.possession == "Turnover") & (match.possession.shift(-1) != "Turnover")

In [53]:
match = match.copy()

In [54]:
match['new_phase'] = False
match.loc[new_phases, 'new_phase'] = True

In [59]:
match['phase_num'] = match.new_phase.cumsum()

In [64]:
match['p'] = match.phase_num % 7

In [65]:
match.head(50)

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,...,pos_orig_x,pos_dest_y,pos_dest_x,team,successful_action,possession,time,new_phase,phase_num,p
378660,8,Simple pass,[{'id': 1801}],15808,"[{'y': 49, 'x': 49}, {'y': 41, 'x': 31}]",2499943,Pass,1612,1H,2.528085,...,68,16,35,Liverpool,True,Liverpool,0:2,False,0,0
378661,8,Simple pass,[{'id': 1801}],25393,"[{'y': 41, 'x': 31}, {'y': 70, 'x': 34}]",2499943,Pass,1612,1H,6.517082,...,32,7,32,Liverpool,True,Liverpool,0:6,False,0,0
378662,8,Simple pass,[{'id': 1801}],14870,"[{'y': 70, 'x': 34}, {'y': 41, 'x': 28}]",2499943,Pass,1612,1H,10.732865,...,36,7,36,Liverpool,True,Liverpool,0:10,False,0,0
378663,8,Simple pass,[{'id': 1801}],25393,"[{'y': 41, 'x': 28}, {'y': 50, 'x': 7}]",2499943,Pass,1612,1H,14.110714,...,33,5,33,Liverpool,True,Liverpool,0:14,False,0,0
378664,8,Launch,[{'id': 1802}],15175,"[{'y': 50, 'x': 7}, {'y': 9, 'x': 61}]",2499943,Pass,1612,1H,16.587550,...,37,81,63,Liverpool,False,Turnover,0:16,False,0,0
378665,6,NaN,[],25747,"[{'y': 9, 'x': 61}, {'y': 0, 'x': 0}]",2499943,Offside,1612,1H,19.464496,...,71,0,29,Liverpool,None,Turnover,0:19,True,1,1
378666,3,Free Kick,[{'id': 1801}],8277,"[{'y': 89, 'x': 35}, {'y': 74, 'x': 18}]",2499943,Free Kick,1625,1H,35.997091,...,100,0,25,Manchester City,True,Manchester City,0:35,False,1,1
378667,8,Simple pass,[{'id': 1801}],9380,"[{'y': 74, 'x': 18}, {'y': 45, 'x': 12}]",2499943,Pass,1625,1H,38.497027,...,48,89,52,Manchester City,True,Manchester City,0:38,False,1,1
378668,8,Simple pass,[{'id': 1801}],71654,"[{'y': 45, 'x': 12}, {'y': 25, 'x': 14}]",2499943,Pass,1625,1H,41.991763,...,55,11,48,Manchester City,True,Manchester City,0:41,False,1,1
378669,8,Simple pass,[{'id': 1801}],70086,"[{'y': 25, 'x': 14}, {'y': 5, 'x': 12}]",2499943,Pass,1625,1H,43.942339,...,45,84,55,Manchester City,True,Manchester City,0:43,False,1,1


In [68]:
cols = ['p', 'phase_num', 'new_phase', 'eventName', 'subEventName', 'team', 'successful_action', 'possession', 'time']

In [69]:
match[cols].to_csv("match.csv")

In [70]:
match.head(25)

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,...,pos_orig_x,pos_dest_y,pos_dest_x,team,successful_action,possession,time,new_phase,phase_num,p
378660,8,Simple pass,[{'id': 1801}],15808,"[{'y': 49, 'x': 49}, {'y': 41, 'x': 31}]",2499943,Pass,1612,1H,2.528085,...,68,16,35,Liverpool,True,Liverpool,0:2,False,0,0
378661,8,Simple pass,[{'id': 1801}],25393,"[{'y': 41, 'x': 31}, {'y': 70, 'x': 34}]",2499943,Pass,1612,1H,6.517082,...,32,7,32,Liverpool,True,Liverpool,0:6,False,0,0
378662,8,Simple pass,[{'id': 1801}],14870,"[{'y': 70, 'x': 34}, {'y': 41, 'x': 28}]",2499943,Pass,1612,1H,10.732865,...,36,7,36,Liverpool,True,Liverpool,0:10,False,0,0
378663,8,Simple pass,[{'id': 1801}],25393,"[{'y': 41, 'x': 28}, {'y': 50, 'x': 7}]",2499943,Pass,1612,1H,14.110714,...,33,5,33,Liverpool,True,Liverpool,0:14,False,0,0
378664,8,Launch,[{'id': 1802}],15175,"[{'y': 50, 'x': 7}, {'y': 9, 'x': 61}]",2499943,Pass,1612,1H,16.587550,...,37,81,63,Liverpool,False,Turnover,0:16,False,0,0
378665,6,NaN,[],25747,"[{'y': 9, 'x': 61}, {'y': 0, 'x': 0}]",2499943,Offside,1612,1H,19.464496,...,71,0,29,Liverpool,None,Turnover,0:19,True,1,1
378666,3,Free Kick,[{'id': 1801}],8277,"[{'y': 89, 'x': 35}, {'y': 74, 'x': 18}]",2499943,Free Kick,1625,1H,35.997091,...,100,0,25,Manchester City,True,Manchester City,0:35,False,1,1
378667,8,Simple pass,[{'id': 1801}],9380,"[{'y': 74, 'x': 18}, {'y': 45, 'x': 12}]",2499943,Pass,1625,1H,38.497027,...,48,89,52,Manchester City,True,Manchester City,0:38,False,1,1
378668,8,Simple pass,[{'id': 1801}],71654,"[{'y': 45, 'x': 12}, {'y': 25, 'x': 14}]",2499943,Pass,1625,1H,41.991763,...,55,11,48,Manchester City,True,Manchester City,0:41,False,1,1
378669,8,Simple pass,[{'id': 1801}],70086,"[{'y': 25, 'x': 14}, {'y': 5, 'x': 12}]",2499943,Pass,1625,1H,43.942339,...,45,84,55,Manchester City,True,Manchester City,0:43,False,1,1
